In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path 
#from rich import inspect

In [69]:
## Build'n'Load Stuff (paths and arrays)
freemocap_validation_data_path = Path(r"C:\Users\aaron\Documents\HumonLab\Spring2022\ValidationStudy\FreeMocap_Data") #path to your data folder
sessionID = 'session_SER_1_20_22' #name of the sessionID folder
this_freemocap_session_path = freemocap_validation_data_path / sessionID
this_freemocap_data_path = this_freemocap_session_path/'DataArrays'
mediapipe_data_path = this_freemocap_data_path / 'mediaPipeSkel_3d_smoothed.npy'


In [70]:

mediapipeSkel_fr_mar_dim = np.load(mediapipe_data_path) #loads in the data as a numpy array

In [71]:
num_pose_joints = 33 #number of pose joints tracked by mediapipe 




pose_joint_range = range(num_pose_joints)
frame_range = range(first_frame,last_frame)

mediapipe_pose_data = mediapipeSkel_fr_mar_dim[:,pose_joint_range[0]:pose_joint_range[-1],:] #load just the pose joints into a data array

num_frames = len(mediapipe_pose_data)

num_frame_range = range(num_frames)

skel_x = mediapipe_pose_data[:,:,0]
skel_y = mediapipe_pose_data[:,:,1]
skel_z = mediapipe_pose_data[:,:,2]

In [72]:

#segments is a list of the body segments that we want the center of mass for. joint_connections is the corresponding joints that make up that segment in order of [proximal joint,distal joint]
# segment_COM_lengths is a corresponding list of center of mass lengths for each segment, and segment_COM_percentages is a corresponding list of the percentage of total body COM each segment takes up. 
segments = [
'right_upper_arm',
'left_upper_arm',
]

joint_connections = [
['right_shoulder','right_elbow'],
['left_shoulder','left_elbow'],
]

segment_COM_lengths = [
.5772,
.5772,
]
segment_COM_percentages = [
.0271,
.0271,
]

num_segments = len(segments)

In [73]:

#create a table with the joint connections, COM length, and percentage for each segment into a dataframe
df = pd.DataFrame(list(zip(segments,joint_connections,segment_COM_lengths,segment_COM_percentages)),columns = ['Segment Name','Joint Connection','Segment COM Length','Segment COM Percentage'])

#create a new copy of this dataframe where the index is set to be the body segment (as opposed to 0,1,2 etc.)
segment_conn_len_perc_dataframe = df.set_index('Segment Name')


In [74]:
def build_mediapipe_skeleton(mediapipe_pose_data,segment_dataframe):
    """ This function takes in the mediapipe pose data array and the segment_conn_len_perc_dataframe. 
        For each frame of data, it loops through each segment we want to find and identifies the names
        of the proximal and distal joints of that segment. Then it searches the mediapipe_indices list
        to find the index that corresponds to the name of that segment. We plug the index into the 
        mediapipe_pose_data array to find the proximal/distal joints' XYZ coordinates at that frame. 
        The segment, its proximal joint and its distal joint gets thrown into a dictionary. 
        And then that dictionary gets saved to a list for each frame. By the end of the function, you 
        have a list that contains the skeleton segment XYZ coordinates for each frame."""
    
    #this is a list of the 33 pose joints that mediapipe tracks
    mediapipe_indices = [
    'nose',
    'right_eye_inner',
    'right_eye',
    'right_eye_outer',
    'left_eye_inner',
    'left_eye',
    'left_eye_outer',
    'right_ear',
    'left_ear',
    'mouth_right',
    'mouth_left',
    'right_shoulder',
    'left_shoulder',
    'right_elbow',
    'left_elbow',
    'right_wrist',
    'left_wrist',
    'right_pinky_1',
    'left_pinky_1',
    'right_index_1',
    'left_index_1',
    'right_thumb_2',
    'left_thumb_2',
    'right_hip',
    'left_hip',
    'right_knee',
    'left_knee',
    'right_ankle',
    'left_ankle',
    'right_heel',
    'left_heel',
    'right_foot_index',
    'left_foot_index'
    ]
  
    mediapipe_frame_segment_joint_XYZ = [] #empty list to hold all the skeleton XYZ coordinates/frame
    for frame in num_frame_range: #NOTE: need to change frame_range to numFrames 2/09/2022 - AC
        mediapipe_pose_skeleton_coordinates = {}
        for segment,segment_info in segment_conn_len_perc_dataframe.iterrows(): #iterate through the data frame by the segment name and all the info for that segment
            proximal_joint_name = segment_info['Joint Connection'][0] 
            distal_joint_name = segment_info['Joint Connection'][1]

            #get the mediapipe index for the proximal and distal joint for this segment
            proximal_joint_index = mediapipe_indices.index(proximal_joint_name)
            distal_joint_index = mediapipe_indices.index(distal_joint_name)

            #use the mediapipe indices to get the XYZ coordinates for the prox/distal joints and throw it in a dictionary
            mediapipe_pose_skeleton_coordinates[segment] = {'proximal':mediapipe_pose_data[frame,proximal_joint_index,:],'distal':mediapipe_pose_data[frame,distal_joint_index,:]}
        mediapipe_frame_segment_joint_XYZ.append(mediapipe_pose_skeleton_coordinates)
        f = 2 
    
    return mediapipe_frame_segment_joint_XYZ

    f= 2
skelcoordinates_frame_segment_joint_XYZ = build_mediapipe_skeleton(mediapipe_pose_data,segment_conn_len_perc_dataframe)
 


f=2 

In [75]:
def calculate_segment_COM(segment_conn_len_perc_dataframe,skelcoordinates_frame_segment_joint_XY):
    segment_COM_frame_dict = []
    for frame in num_frame_range:
        segment_COM_dict = {}
        for segment,segment_info in segment_conn_len_perc_dataframe.iterrows():

            this_segment_XYZ = skelcoordinates_frame_segment_joint_XY[frame][segment]
            this_segment_proximal = this_segment_XYZ['proximal']
            this_segment_distal = this_segment_XYZ['distal']
            this_segment_COM_length = segment_info['Segment COM Length']

            this_segment_COM = this_segment_proximal + this_segment_COM_length*(this_segment_distal-this_segment_proximal)
            
            segment_COM_dict[segment] = this_segment_COM
        segment_COM_frame_dict.append(segment_COM_dict)
    return segment_COM_frame_dict
    f = 2
segment_COM_frame_dict = calculate_segment_COM(segment_conn_len_perc_dataframe, skelcoordinates_frame_segment_joint_XYZ)

def reformat_segment_COM(segment_COM_frame_dict):
    
    test_array = np.empty([int(len(num_frame_range)),int(num_segments),3])
    for frame in num_frame_range:
        this_frame_skeleton = segment_COM_frame_dict[frame]
        for joint_count,segment in enumerate(this_frame_skeleton.keys()):
            test_array[frame,joint_count,:] = this_frame_skeleton[segment]
    return test_array

segment_COM_frame_imgPoint_XYZ = reformat_segment_COM(segment_COM_frame_dict)

f=2

            



In [76]:
good_frame = 6000
goodframe_x = skel_x[good_frame,:]
goodframe_y = skel_y[good_frame,:]
goodframe_z = skel_z[good_frame,:]

segment_COM_x = segment_COM_frame_imgPoint_XYZ[good_frame,:,0]
segment_COM_y = segment_COM_frame_imgPoint_XYZ[good_frame,:,1]
segment_COM_z = segment_COM_frame_imgPoint_XYZ[good_frame,:,2]



def calculate_total_COM()

In [77]:
%matplotlib
#plt.ion()
figure = plt.figure()
ax = figure.add_subplot(111, projection = '3d')
ax.scatter(goodframe_x, goodframe_y,goodframe_z)
ax.scatter(segment_COM_x,segment_COM_y,segment_COM_z)
plt.show()
# inspect(ax,methods= True)

Using matplotlib backend: TkAgg


In [25]:
# #http://www.kdm.p.lodz.pl/articles/2017/3/21_3_4.pdf
# r_upperarm_COM_x = goodframe_x[r_upperarm[0]] + (.5772)*(goodframe_x[r_upperarm[1]]-goodframe_x[r_upperarm[0]])
# l_upperarm_COM_x = goodframe_x[l_upperarm[0]] + (.5772)*(goodframe_x[l_upperarm[1]]-goodframe_x[l_upperarm[0]])

# r_forearm_COM_x = goodframe_x[r_forearm[0]] + (.4574)*(goodframe_x[r_forearm[1]]-goodframe_x[r_forearm[0]])
# l_forearm_COM_x = goodframe_x[l_forearm[0]] + (.4574)*(goodframe_x[l_forearm[1]]-goodframe_x[l_forearm[0]])

# r_upperarm_COM_y = goodframe_y[r_upperarm[0]] + (.5772)*(goodframe_y[r_upperarm[1]]-goodframe_y[r_upperarm[0]])
# l_upperarm_COM_y = goodframe_y[l_upperarm[0]] + (.5772)*(goodframe_y[l_upperarm[1]]-goodframe_y[l_upperarm[0]])

# r_forearm_COM_y = goodframe_y[r_forearm[0]] + (.4574)*(goodframe_y[r_forearm[1]]-goodframe_y[r_forearm[0]])
# l_forearm_COM_y = goodframe_y[l_forearm[0]] + (.4574)*(goodframe_y[l_forearm[1]]-goodframe_y[l_forearm[0]])

# segment_com_x = [r_upperarm_COM_x,l_upperarm_COM_x,r_forearm_COM_x,l_forearm_COM_x]
# segment_com_y = [r_upperarm_COM_y,l_upperarm_COM_y,r_forearm_COM_y,l_forearm_COM_y]

